# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,college,64.8569,-147.8028,-13.01,82,40,2.57,US,1730083742
1,1,hilton head island,32.2163,-80.7526,18.88,70,100,5.66,US,1730083743
2,2,margaret river,-33.9500,115.0667,20.94,48,0,2.88,AU,1730083744
3,3,kristinestad,62.2743,21.3760,8.06,76,2,5.69,FI,1730083746
4,4,borodulikha,50.7184,80.9295,8.58,77,100,6.86,KZ,1730083748


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    alpha = 0.5
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df[(city_data_df['Max Temp'] > 21) & (city_data_df['Max Temp'] < 27)
                        & (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
ideal_df.dropna()

# Display sample data
ideal_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
51,51,port isabel,26.0734,-97.2086,24.02,78,0,3.09,US,1730083806
78,78,maintirano,-18.0667,44.0167,25.02,81,0,1.39,MG,1730083838
98,98,lazaro cardenas,17.9583,-102.2000,26.68,79,0,1.27,MX,1730083861
226,226,shendi,16.6915,33.4341,26.72,17,0,3.92,SD,1730084014
251,251,alaghsas,17.0187,8.0168,24.06,19,0,4.32,NE,1730084045


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df.copy()
hotel_df = hotel_df[['City','Country','Lat','Lng','Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
51,port isabel,US,26.0734,-97.2086,78,
78,maintirano,MG,-18.0667,44.0167,81,
98,lazaro cardenas,MX,17.9583,-102.2000,79,
226,shendi,SD,16.6915,33.4341,17,
251,alaghsas,NE,17.0187,8.0168,19,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "radius":radius,
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port isabel - nearest hotel: Queen Isabel Inn
maintirano - nearest hotel: Hotel Safari
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
shendi - nearest hotel: Hotel - unknown name
alaghsas - nearest hotel: Hôtel Zagado
tura - nearest hotel: Hotel Polo Orchid
gulf shores - nearest hotel: Beachside Resort
truth or consequences - nearest hotel: Travelodge
saint-pierre - nearest hotel: Tropic Hotel
tepalcatepec - nearest hotel: No hotel found
villa juarez - nearest hotel: No hotel found
bushehr - nearest hotel: هتل رستوران سیراف
phongsali - nearest hotel: Phonsaly Hotel
brandon - nearest hotel: Brandon Motor Lodge
piton saint-leu - nearest hotel: Appartement Piton Saint Leu


,City,Country,Lat,Lng,Humidity,Hotel Name
51,port isabel,US,26.0734,-97.2086,78,Queen Isabel Inn
78,maintirano,MG,-18.0667,44.0167,81,Hotel Safari
98,lazaro cardenas,MX,17.9583,-102.2000,79,Hotel Sol del Pacífico
226,shendi,SD,16.6915,33.4341,17,Hotel - unknown name
251,alaghsas,NE,17.0187,8.0168,19,Hôtel Zagado
264,tura,IN,25.5198,90.2201,67,Hotel Polo Orchid
280,gulf shores,US,30.2460,-87.7008,1,Beachside Resort
285,truth or consequences,US,33.1284,-107.2528,17,Travelodge
360,saint-pierre,RE,-21.3393,55.4781,69,Tropic Hotel
409,tepalcatepec,MX,19.1833,-102.8500,63,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    alpha = 0.5,
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name","Country"]
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)